In [2]:
import breakwater as bw
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [3]:
from development_overtopping_DKA import (eurotop2018_6_5, 
                                         surf_similarity, 
                                         gamma_beta_eurotop_2018_6_9, 
                                         calc_beta)
from development_armour_stability_DKA import (vangent_armour_reduction, 
                                              hudson_fixed_slope, 
                                              rock_manual_5_196, 
                                              rock_manual_5_195, 
                                              rock_manual_5_194)
from development_material_DKA import(get_class,
                                     get_Dn50)

from development_scour_DKA import(sumer_fredsoe,
                                 v_scour)


from breakwater.utils.exceptions import user_warning

In [38]:
project_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                             index_col = 1,
                             sheet_name='Input_Project specific')
requirements_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                                  index_col = 0,
                                  sheet_name='Input_requirements')
wave_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                          index_col = 0,
                          sheet_name='input_hydrotechnical',
                         skiprows = 1)
cross_section_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                   sheet_name='Input_Cross section',
                                  skiprows = 1)
concrete_element_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                      sheet_name='Input_concrete_elements',
                                      index_col = 0,
                                      skiprows = 1)
gradings_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                      sheet_name='input_rock_gradings',
                                      index_col = 0,
                                      skiprows = 2)

C:\ProgramData\anaconda3\envs\env_bw\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [36]:
#project_data

In [47]:
#requirements_data

In [48]:
#wave_data

In [49]:
#cross_section_data

In [39]:
gradings_data

,ELL,NLL,NUL,EUL,"M_em,LL","M_em,M","M_em,UL",nrrm,M50/Mem,"M50,LL","M50,m","M50,UL",Client range,Unnamed: 14,Dn50 av,Layer thickness,W_M50,Dn50 min,Armour underlayer,Toe underlayer
5-40 kg,1.5,5,40,80,10,15.0,20,1.74,1.386,13.86,20.790,27.72,10,20,0.199968,0.363942,13.86,0.174688,1-500 kg,63-180 mm
10-60 kg,2.0,10,60,120,20,27.5,35,2.02,1.352,27.04,37.180,47.32,20,35,0.242724,0.441757,27.04,0.218279,1-500 kg,63-180 mm
40-200 kg,15.0,40,200,300,80,100.0,120,2.25,1.269,101.52,126.900,152.28,80,120,0.365451,0.665121,101.52,0.339255,1-500 kg,63-180 mm
60-300 kg,30.0,60,300,450,120,155.0,190,2.25,1.243,149.16,192.665,236.17,130,190,0.420026,0.764448,149.16,0.385679,1-500 kg,63-180 mm
0.3-1 ton,200.0,300,1000,1500,540,615.0,690,3.00,1.163,628.02,715.245,802.47,540,690,0.650370,1.183673,628.02,0.622778,1-500 kg,63-180 mm
1-3 ton,700.0,1000,3000,4500,1700,1900.0,2100,3.29,1.099,1868.30,2088.100,2307.90,1700,2100,0.929521,1.691729,1868.30,0.895690,1-500 kg,40-200 kg
3-6 ton,2000.0,3000,6000,9000,4200,4500.0,4800,5.22,1.054,4426.80,4743.000,5059.20,4200,4800,1.221874,2.223812,4426.80,1.194095,40-200 kg,40-200 kg
6-10 ton,4000.0,6000,10000,15000,7500,8000.0,8500,7.08,1.024,7680.00,8192.000,8704.00,7500,8500,1.466017,2.668150,7680.00,1.434815,NaN,NaN
10-15 ton,6500.0,10000,15000,22500,12000,12500.0,13000,8.92,1.002,12024.00,12525.000,13026.00,12000,13000,1.688890,3.073780,12024.00,1.666065,NaN,NaN


In [18]:
# CALCULATE CREST HEIGHT FOR OVERTOPPING

# Do we want to put this entire part in a function to keep the scripts clean a bit?


Calculation_case = 57
Cross_section_id = 0
# Public access: allowed

# Open project specific parameters
g = project_data.at['g', 'Value']

# Open structure specific parameters
tana          = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure = cross_section_data.at[Cross_section_id, 'dir_structure']
safety        = cross_section_data.at[Cross_section_id, 'safety']

# Get info for sea state. Currently implemented to do only the first sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']

# Get allowed q for the location and calculation case
LS        = wave_data.at[Calculation_case, 'Limit State']
q_allowed = requirements_data.at['Overtopping limit public access', LS]


#Calculate roughness reduction
# Set armour for rougness reduction
armour_layer = 'Rock'
layers       = 2
permeability = 'permeable'

# Calculate surf-similarity with Hm0 and Tm-1,0
xi_m_min_1 = surf_similarity(tana, Hm0, Tm_min_1, g)

gamma_f = bw.core.overtopping.gamma_f(armour_layer, 
                                      xi_m_min_1, 
                                      layers = layers, 
                                      permeability = permeability) 

#Calculate obliqueness reduction. Function based on SAWP-#3504459-V48-IHS-COA-xxx-CAL_Armour_Stability_under_Waves.XLSM
beta = calc_beta(dir_structure, dir_wave)
gamma_beta = gamma_beta_eurotop_2018_6_9(beta)


Rc = eurotop2018_6_5(g, Hm0, q_allowed, gamma_f, gamma_beta, limit = False, safety=safety)
z_crest = Rc + wl

In [19]:
print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
print("q_allowed : "+str(q_allowed))
#print("g         : "+str(g))
#print("Hm0       : "+str(Hm0))
#print("Tm_min_1  : "+str(Tm_min_1))
#print("wl        : "+str(wl))
print("xi_m_min_1: "+str(xi_m_min_1))
print("Rc        : "+str(Rc))
print("beta      : "+str(beta))
print("gamma_f   : "+str(gamma_f))
print("gamma_beta: "+str(gamma_beta))
print("z_crest   : "+str(z_crest))


Cross-section : ECn2 - 370N
Wave info for : ECn2 - 370N
q_allowed : 5.0
xi_m_min_1: 3.545428517642052
Rc        : 1.254756265426078
beta      : 65.5
gamma_f   : 0.4
gamma_beta: 0.58735
z_crest   : 1.744756265426078


In [10]:
# CALCULATE REQUIRED STONE DIAMETER (Rock armour)

Calculation_case = 57
Cross_section_id = 0
# Armour = 2 layers rock, permeable core

# Open project specific parameters
g              = project_data.at['g'             , 'Value']
rho_a          = project_data.at['rho_r'         , 'Value']
rho_w          = project_data.at['rho_w'         , 'Value']
Cpl_shallow    = project_data.at['Cpl_shallow'   , 'Value']
Cs_shallow     = project_data.at['Cs_shallow'    , 'Value']
c_beta         = project_data.at['c_beta'        , 'Value']
c_rh           = project_data.at['c_rh'          , 'Value']
beta_max       = project_data.at['beta_max'      , 'Value']
Storm_duration = project_data.at['storm_duration', 'Value']
Safety         = project_data.at['sf_vdm'        , 'Value'] #Safety factor on top of regular VDM constants

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']
Tm_0_2   = wave_data.at[Calculation_case, 'Tm0,2']
os_bin   = wave_data.at[Calculation_case, 'Offshore bin']
N        = np.round(Storm_duration*3600/Tm_0_2)


# Open structure specific parameters
tana            = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure   = cross_section_data.at[Cross_section_id, 'dir_structure']
safety          = cross_section_data.at[Cross_section_id, 'safety']
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']
slope_foreshore = cross_section_data.at[Cross_section_id, 'slope_foreshore']
P               = cross_section_data.at[Cross_section_id, 'P']
rh              = cross_section_data.at[Cross_section_id, 'roundhead']


# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Sd_allowed = requirements_data.at['Armour damage limit', LS]

#Intermediate calculations
h              = wl-z_bed
Delta          = (rho_a-rho_w)/rho_w
alpha          = np.arctan(tana)
NEN_gradings   = bw.RockGrading(rho=rho_a)
waveinfo       = bw.BattjesGroenendijk(Hm0, h, slope_foreshore)
H2_per         = waveinfo.get_Hp(0.02)
Hs             = waveinfo.get_Hn(3) #NU BATTJES-GROENENDIJK VOOR Hs UIT Hmo. IS DAT WAT WE WILLEN?

#OVERRIDE wave characteristics BECAUSE W+B sheet tales Hs = Hm0
Hs = Hm0
#H2_per = Hs*1.34
#user_warning(f"Hs = Hm0, H2_per taken from W+B calculation due to inconsistencies in Battjes Groenendijk")
xi_s_0_2     = surf_similarity(tana, Hs, Tm_0_2, g)

# Check validity of Van der Meer shallow
vdm_shallow_validity = h/Hs

#Calculate required stone diameter without reduction
Dn50 = bw.core.vandermeer_shallow(Hs, 
                                  H2_per, 
                                  Delta, 
                                  P, 
                                  Sd_allowed, 
                                  N, 
                                  xi_s_0_2, 
                                  alpha, 
                                  Cpl = Cpl_shallow, 
                                  Cs = Cs_shallow, 
                                  safety = Safety)

#Reduce Dn50 with reduction factors from DAR

#Calculate obliqueness reduction. Function based on SAWP-#3504459-V48-IHS-COA-xxx-CAL_Armour_Stability_under_Waves.XLSM
beta = calc_beta(dir_structure, dir_wave)
gamma_beta  = vangent_armour_reduction(beta, c_beta, beta_max)
Dn50_oblique = Dn50*gamma_beta

#Dn50_roundhead, no obliqueness correction on roundhead
Dn50_rh = Dn50*c_rh 

#Select stone size to apply
if rh == "Yes":
    Dn50_selected = Dn50_rh
elif rh == "No":
    Dn50_selected = Dn50_oblique
else:
    user_warning(f"Incorrect input in Roundhead indication")
    Dn50_selected = 10000

# Select grading    
#grading = NEN_gradings.get_class(Dn50 = Dn50_selected) 
#It looks like this will only go to the next class if M50>M50_max for a class. Is this the behaviour we want?

In [11]:
#print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
#print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
#print("Limit State   : "+str(LS))
#print("Bin           : "+str(os_bin))
#print("Allowed Sd    : "+str(Sd_allowed))
#print("g         : "+str(g))
print("h         : "+str(h))
print("Delta     : "+str(Delta))
print("P         : "+str(P))
#print("Tangent a : "+str(tana))
#print("Hm0       : "+str(Hm0))
print("Hs        : "+str(Hs))
print("H2%       : "+str(H2_per))
print("H2%/Hs    : "+str(H2_per/Hs))
#print("Tm0,2     : "+str(Tm_0_2))
print("N         : "+str(N))
print("xi_s_0_2  : "+str(xi_s_0_2))
print("Dn50      : "+str(Dn50))
print("h/Hs (<3) : "+str(vdm_shallow_validity))
#print("beta      : "+str(beta))
print("gamma_beta: "+str(gamma_beta))
print("Dn50_oblique: "+str(Dn50_oblique))
print("Dn50_rh    : "+str(Dn50_rh))
print("Dn50_select: "+str(round(Dn50_selected,4)))
print("Grading    : "+str(grading))


h         : 7.1000000000000005
Delta     : 1.5742574257425743
P         : 0.5
Hs        : 2.24
H2%       : 2.9997113157614903
H2%/Hs    : 1.3391568373935223
N         : 4821.0
xi_s_0_2  : 2.493488188231773
Dn50      : 0.7964916534126327
h/Hs (<3) : 3.169642857142857
gamma_beta: 0.46178081557808515
Dn50_oblique: 0.36780456531402306
Dn50_rh    : 0.8578215107254055
Dn50_select: 0.3678
Grading    : LMA_60/300


In [12]:
concrete_element_data


,Nod,Accropode II-trunk,Accropode II-roundhead,Xbloc-trunk,Xbloc-roundhead
Limit State,,,,,
SLS,0.0,2.8,2.5,2.77,2.57
ULS,0.5,3.1,2.8,3.10,2.90


In [13]:
# Calculate concrete armour stability

Calculation_case = 57
Cross_section_id = 0

# Open project specific parameters
rho_c  = project_data.at['rho_c'      , 'Value'] #For rock armour. Needs adaptation for concrete
rho_w  = project_data.at['rho_w'      , 'Value']
unit   = project_data.at['c_unit'     , 'Value']

# Get info for sea state
LS     = wave_data.at[Calculation_case, 'Limit State']
Hm0    = wave_data.at[Calculation_case, 'Hm0']

# Open cross-section specific parameters
rh     = cross_section_data.at[Cross_section_id, 'roundhead']

# Intermediate calculations
Hs = Hm0 #TO BE CHANGED DEPENDING ON DESIGN APPROACH
Delta = (rho_c-rho_w)/rho_w


# Create stability case to find outcome for Hudson relationship
if rh == "Yes":
    stability_case = unit+"-roundhead"
elif rh == "No":
    stability_case = unit+"-trunk"
else:
    user_warning(f"Incorrect input in Roundhead identifier")
    stability_case = None

Hudson_outcome = concrete_element_data.at[LS, stability_case]    

Dn50_concrete = hudson_fixed_slope(Hs, Hudson_outcome, Delta)
V_unit = Dn50_concrete**3

In [15]:
#print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
#print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
#print("Limit State   : "+str(LS))
#print("rho_c         : "+str(rho_c))
#print("rho_w         : "+str(rho_w))
print("Delta         : "+str(Delta))
print("Unit          : "+str(unit))
print("Roundhead     : "+str(rh))
print("Hs            : "+str(Hs))
print("Hudson outcome: "+str(Hudson_outcome))
print("Dn50_units    : "+str(Dn50_concrete))
print("V_units       : "+str(V_unit))


Delta         : 1.3267326732673268
Unit          : Xbloc
Roundhead     : No
Hs            : 2.24
Hudson outcome: 2.77
Dn50_units    : 0.6095155989008029
V_units       : 0.22644069233860178


In [20]:
# CALCULATE REAR SIDE ARMOUR STABILITY
# This calculation needs a crest height as input and therefore needs to be done after calculation of 
# the required crest height. If the calculation is done before choosing the final crest height it can give conservative
# results since the crest height in the calculation is lower than the final crest height.
# The calculation can also be performed in the verification stage only.

#To-do
# Hs-Hm0 interpretation
# Select correct Rc
# Adapt grading selection behaviour

Calculation_case = 57
Cross_section_id = 0
# Armour = 2 layers rock, permeable core

# Open project specific parameters
g               = project_data.at['g'              , 'Value']
rho_a           = project_data.at['rho_r'          , 'Value']
slope_roughness = project_data.at['slope_roughness', 'Value']
Storm_duration  = project_data.at['storm_duration' , 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']
Tm_0_2   = wave_data.at[Calculation_case, 'Tm0,2']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']

# Open structure specific parameters
tana            = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure   = cross_section_data.at[Cross_section_id, 'dir_structure']
cota_rear       = cross_section_data.at[Cross_section_id, 'cot_a_rear']
crest_roughness = cross_section_data.at[Cross_section_id, 'crest_roughness']
B               = cross_section_data.at[Cross_section_id, 'B']

# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Sd_allowed = requirements_data.at['Armour damage limit', LS]

#Intermediate calculations
h              = wl-z_bed
beta = calc_beta(dir_structure, dir_wave)
waveinfo       = bw.BattjesGroenendijk(Hm0, h, slope_foreshore)
H2_per         = waveinfo.get_Hp(0.02)
Hs             = waveinfo.get_Hn(3) #NU BATTJES-GROENENDIJK VOOR Hs UIT Hmo. IS DAT WAT WE WILLEN?

#OVERRIDE wave characteristics BECAUSE W+B sheet tales Hs = Hm0
Hs = Hm0 #TO BE ADAPTED IF WE CHOOSE TO DIFFERENTIATE BETWEEN Hs AND Hm0

Delta          = (rho_a-rho_w)/rho_w
xi_s_min_1     = surf_similarity(tana, Hs, Tm_min_1, g)

NEN_gradings   = bw.RockGrading(rho=rho_a)

# Actual calculations
Ru_1_percent, gamma_f, gamma_beta, gamma  = rock_manual_5_196(Hs, xi_s_min_1, slope_roughness, beta)
u_1_percent = rock_manual_5_195(g, slope_roughness, crest_roughness, Ru_1_percent, Rc, B, Hs)
#Rc IS NOW JUST THE PRESENT Rc. THIS NEEDS TO BE COUPLED TO CALCULATION CASE
Dn50_rear = rock_manual_5_194(Sd_allowed, N, u_1_percent, Tm_min_1, Delta, cota_rear, Rc, Hs)

# Select grading    
#grading_rear = NEN_gradings.get_class(Dn50 = Dn50_rear) 
#It looks like this will only go to the next class if M50>M50_max for a class. Is this the behaviour we want?

In [22]:
print("Cross-section  : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
print("Wave info for  : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
print("Limit State    : "+str(LS))
print("Hs             : "+str(Hs))
print("xi_s_min_1     : "+str(xi_s_min_1))
print("slope_roughness: "+str(slope_roughness))
print("crest_roughness: "+str(crest_roughness))
print("beta           : "+str(beta))
print("gamma_f        : "+str(gamma_f))
print("gamma_beta     : "+str(gamma_beta))
print("gamma          : "+str(gamma))
print("Ru_1_percent   : "+str(Ru_1_percent))
print("Rc             : "+str(Rc))
print("B              : "+str(B))
print("u_1_percent    : "+str(u_1_percent))
print("Sd_allowed     : "+str(Sd_allowed))
print("N              : "+str(N))
print("Tm_min_1       : "+str(Tm_min_1))
print("Delta          : "+str(Delta))
print("cota_rear:     : "+str(cota_rear))
print("Dn50_rear      : "+str(Dn50_rear))
#print("Grading rear   : "+str(grading_rear))

Cross-section  : ECn2 - 370N
Wave info for  : ECn2 - 370N
Limit State    : SLS
Hs             : 2.24
xi_s_min_1     : 3.545428517642052
slope_roughness: rough
crest_roughness: rough
beta           : 65.5
gamma_f        : 0.55
gamma_beta     : 0.8559
gamma          : 0.470745
Ru_1_percent   : 4.044031891608836
Rc             : 1.254756265426078
B              : 6
u_1_percent    : 7.013887880646336
Sd_allowed     : 2.0
N              : 4821.0
Tm_min_1       : 6.37
Delta          : 1.5742574257425743
cota_rear:     : 1.5
Dn50_rear      : 0.5968088741544244


In [40]:
# CALCULATE TOE STABILITY

Calculation_case = 57
Cross_section_id = 0

# Open project specific parameters
g            = project_data.at['g'           , 'Value']
rho_a        = project_data.at['rho_r'       , 'Value']
rho_w        = project_data.at['rho_w'       , 'Value']
Lt           = project_data.at['Lt'          , 'Value']
t_underlayer = project_data.at['t_underlayer', 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']

# Open structure specific parameters
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']

# Get allowed Sd for the location and calculation case
LS          = wave_data.at[Calculation_case, 'Limit State']
Nod_allowed = requirements_data.at['Toe damage limit', LS]

# Intermediate parameters
h               = wl-z_bed
Delta          = (rho_a-rho_w)/rho_w

#TEMPORARY FIX TO BE IN LINE WITH PHASE II
Hs = Hm0

# Calculate required stone size for toe

# use while loop since ht depends on Dn50 of the toe
# first set temporary values for the while loop
Dn50_toe = 0
Dn50_toe_temp = 0
compute_toe = True
counter = 0
t_filter = 0

toe_layer_thickness = t_underlayer + t_filter + Dn50_toe*Lt
ht = h - toe_layer_thickness

while compute_toe:
    Dn50_toe_computed = bw.core.toe_stability(Hs, 
                                              h, 
                                              ht, 
                                              Delta, 
                                              Nod_allowed)    

    # check for convergence
    if Dn50_toe_computed - Dn50_toe_temp < 0 or counter > 50:
        # value has converged, so break loop
        compute_toe = False

    counter += 1
    
    # check if computed Dn50 is larger than current normative Dn50
    if Dn50_toe_computed > Dn50_toe:
        # if larger the normative Dn50 must be changed
        Dn50_toe = Dn50_toe_computed
        
    # Set Dn50 toe class average
    class_toe = get_class(Dn50_toe_computed, rho_a, gradings_data)
    Dn50_toe_computed = get_Dn50(class_toe, gradings_data)
    
    # Check if filter layer needs to be applied
    if gradings_data.at[class_toe, 'Toe underlayer'] in gradings_data.index:
        class_filter = gradings_data.at[class_toe, 'Toe underlayer']
        Dn50_filter = get_Dn50(class_filter, gradings_data)
        t_filter = Dn50_filter*toe_layers
    
    # replace old values with the new ones
    Dn50_toe_temp = Dn50_toe_computed
    toe_layer_thickness = t_underlayer + t_filter + Dn50_toe_temp*Lt
    ht = h - toe_layer_thickness
    
toe_armour_class = get_class(Dn50_toe, rho_a, gradings_data)

In [41]:
print("Cross-section  : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
print("Wave info for  : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
print("Limit State    : "+str(LS))
print("Nod allowed    : "+str(Nod_allowed))
print("Delta          : "+str(Delta))
print("Hs             : "+str(Hs))
print("h              : "+str(h))
print("t_filter       : "+str(t_filter))
print("toe_layer_thickness: "+str(toe_layer_thickness))
print("ht             : "+str(ht))
print("Dn50_toe       : "+str(Dn50_toe))
print("toe_layer_class: "+str(toe_armour_class))
print("Class Dn50     : "+str(Dn50_toe_temp))

Cross-section  : ECn2 - 370N
Wave info for  : ECn2 - 370N
Limit State    : SLS
Nod allowed    : 0.5
Delta          : 1.5742574257425743
Hs             : 2.24
h              : 7.1000000000000005
t_filter       : 0
toe_layer_thickness: 1.3651212336680056
ht             : 5.734878766331995
Dn50_toe       : 0.28792068539584126
toe_layer_class: 40-200 kg
Class Dn50     : 0.3654512272901131


In [10]:
# CALCULATE SCOUR DEPTH

Calculation_case = 57
Cross_section_id = 0


# Open project specific parameters
g            = project_data.at['g'           , 'Value']
C2_sf        = project_data.at['C2_sf'       , 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
Tp       = wave_data.at[Calculation_case, 'Tp']
wl       = wave_data.at[Calculation_case, 'wl']

# Open structure specific parameters
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']

# Intermediate parameters
h               = wl-z_bed

#TEMPORARY FIX TO BE IN LINE WITH PHASE II
Hs = Hm0

# Calculate scour depth
S = sumer_fredsoe(Hs, Tp, g, h, C2_sf)

In [11]:
print("Cross-section  : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
print("Wave info for  : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))    
print("Hs    : "+str(Hs))
print("Tp    : "+str(Tp))
print("h     : "+str(h))
print("C2    : "+str(C2_sf))
print("S     : "+str(S))

Cross-section  : ECn2 - 370N
Wave info for  : ECn2 - 370N
Hs    : 2.24
Tp    : 8.14
h     : 7.1000000000000005
C2    : 1
S     : 0.279083956914683


In [33]:
# CALCULATE TOE DIMENSIONS - TO BE PERFORMED ONLY AFTER SELECTING TOE Dn50

# Temporary set to previous value. Still needs to be incorporated after calculating norming Dn50
Dn50_toe = 0.28792068539584126

# Get S for norming case?
S

# Import project specific parameters
Lt_fa      = project_data.at['Lt_fa'     , 'Value']
ltoe_min   = project_data.at['ltoe_min'  , 'Value']
htoe       = project_data.at['htoe'      , 'Value']
cota_scour = project_data.at['cota_scour', 'Value']
rho_a      = project_data.at['rho_r'     , 'Value']

# Choose selected Dn50_toe for this location
toe_armour_class = get_class(Dn50_toe, rho_a, gradings_data)
Dn50_toe = get_Dn50(toe_armour_class, gradings_data)

# Calculate layer thickness for the falling apron
t_fa = Dn50_toe*Lt_fa

# Calculate necessary volumes
Vmin = ltoe_min*Dn50_toe * htoe*Dn50_toe
Vscour = v_scour(S, t_fa, cota_scour)
Vtotal = Vmin + Vscour

# Calculate toe length
Ltoe = Vtotal / (htoe*Dn50_toe)

In [35]:
print(f'toe_armour_class : {toe_armour_class}')
print(f'Dn50_toe : {Dn50_toe}')
print(f'S        : {S}')
print(f't_fa     : {t_fa}')
print(f'Vmin     : {Vmin}')
print(f'Vscour   : {Vscour}')
print(f'Vtotal   : {Vtotal}')
print(f'Ltoe     : {Ltoe}')

toe_armour_class : 40-200 kg
Dn50_toe : 0.3654512272901131
S        : 0.279083956914683
t_fa     : 0.3654512272901131
Vmin     : 0.8013275971670993
Vscour   : 0.32252567779269986
Vtotal   : 1.1238532749597991
Ltoe     : 1.5376241630016874
